# Initialization and loading basic functions  

In [1]:
import os
import numpy as np
import codecs
import json
import string
import random
import itertools
from datetime import datetime, date, timedelta
import mysql.connector
from mysql.connector import errorcode

DB_NAME = "TaxiTest"
config = {
        'user': 'root',
        'password': '9090',
        'host': '127.0.0.1',
        'database': 'taxitest',
        'raise_on_warnings': True,
        'use_pure': False
}

def get_table_review(table, only_columns_names = False):
    print(table)
    print("Field, Type, Null, Key, Default, Extra\n")
    columns_names = []

    q = "DESCRIBE " + table
    cursor.execute(q)
    columns_attributes = cursor.fetchall()
    for column_attributes_tuple in columns_attributes:
        column_attributes = list(column_attributes_tuple)
        column_attributes[1] = str(column_attributes[1])[2:-1]
        columns_names.append(column_attributes[0])
        if not only_columns_names:
            print(', '.join([str(a) for a in column_attributes]))
    
    joined = ', '.join(columns_names)
    joined_values = ', '.join('%s' for c in columns_names)
    print("\nColumns: (", joined, ")")
    insert_string = "query = \"INSERT INTO %s (%s) VALUES (%s)\"" % (table, joined, joined_values)
    print(insert_string)
    
def count_rows(table):
    q = "SELECT COUNT(*) FROM " + table
    cursor.execute(q)
    log = cursor.fetchall()
    print("Total number of rows %s: %s." % (table, log))
    return int(log[0][0])

def delete_all_rows(table):
    q = "DELETE FROM " + table
    cursor.execute(q) 
    
def select_rows(q, table=None):
    cursor.execute(q)
    return cursor.fetchall()

def insert_many_rows(q, data, table=None):
    try:
        cursor.executemany(q, data)
        log = cursor.fetchall()
        print("Result: " + log)
    except mysql.connector.Error as err:
        print(err.msg)

cnx = mysql.connector.connect(**config)
cnx.autocommit = True
cursor = cnx.cursor()

query = "SHOW DATABASES"
cursor.execute(query)
rows = cursor.fetchall()
print("Total number of databases:", cursor.rowcount)
for row in rows:
    print(row)

Total number of databases: 7
('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('taxitest',)
('world',)


In [18]:
# DESCRIBE taxitest.order;

cursor.execute("USE " + DB_NAME)
cursor.execute("SHOW Tables")
tables = cursor.fetchall()


for table in tables:
    print("Table:", str(table[0]))
    #get_table_review(table[0])


Table: address
Table: car
Table: client
Table: driver
Table: insurance
Table: insurancecompany
Table: operator
Table: order
Table: payment
Table: paymentservice
Table: phonecall
Table: taxi


# Start

## Table `insurancecompany`

In [13]:
table = DB_NAME + "." + "insurancecompany"
get_table_review(table)

TaxiTest.insurancecompany
Field, Type, Null, Key, Default, Extra

idInsuranceCompany, int(11), NO, PRI, None, 
Address, varchar(45), NO, , None, 
Phone, varchar(45), NO, , None, 
Mail, varchar(45), NO, , None, 
Name, varchar(45), NO, , None, 

Columns: ( idInsuranceCompany, Address, Phone, Mail, Name )
query = "INSERT INTO TaxiTest.insurancecompany (idInsuranceCompany, Address, Phone, Mail, Name) VALUES (%s, %s, %s, %s, %s)"


In [7]:
count_rows(table)

Total number of rows TaxiTest.insurancecompany: [(30,)].


30

In [ ]:
delete_all_rows(table)

In [ ]:
# Data generation

with open('InsuranceCompanies/Companies.json') as f:
    names = json.load(f)

total_number = len(names)
print("len(names): ", len(names))
idInsuranceCompany = np.arange(total_number)

with open('Streets.json') as f:
    streets = json.load(f)
    streets = np.random.choice(streets, total_number, replace=False)

query = "INSERT INTO insurancecompany (idInsuranceCompany, Address, Phone, Mail, Name) VALUES (%s, %s, %s, %s, %s)"
data = []

for i in range(total_number):
    idInsuranceCompany = i + 1
    address = "Kazan, " + streets[i] + ", " + str(np.random.randint(1, 200))
    phone = '+7(843)' + '{:07}'.format(np.random.randint(0, 9999999))
    name = names[i]
    mail = name.lower().replace(" ", "") + '@gmail.com'
    data.append((idInsuranceCompany, address, phone, mail, name))

print("Q: ", query)
print("D: ", data[0])

insert_many_rows(table, query, data)

## Table `insurance`

*** Changes ***
~~~~python
# Modify type
cursor.execute("ALTER TABLE " + table + " MODIFY InsuranceStartDate DATE")
cursor.execute("ALTER TABLE " + table + " MODIFY InsuranceEndDate DATE")

# Change order
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN InsuranceEndDate InsuranceEndDate Date AFTER InsuranceStartDate")
~~~~

In [ ]:
# Getting InsuranceCompanies list and saving in json 
file = "InsuranceCompanies\Companies.txt"

names = []
with codecs.open(file, 'r', encoding='utf-8') as f:
    for line in f:
        names.append(line[:-2])
        
with open('InsuranceCompanies\Companies.json', 'w') as outfile:
    json.dump(names, outfile)

In [14]:
table = DB_NAME + "." + "insurance"
get_table_review(table)

TaxiTest.insurance
Field, Type, Null, Key, Default, Extra

idInsurance, int(11), NO, PRI, None, 
InsCompany_idInsCompany, int(11), NO, MUL, None, 
InsuranceStartDate, date, YES, , None, 
InsuranceEndDate, date, YES, , None, 

Columns: ( idInsurance, InsCompany_idInsCompany, InsuranceStartDate, InsuranceEndDate )
query = "INSERT INTO TaxiTest.insurance (idInsurance, InsCompany_idInsCompany, InsuranceStartDate, InsuranceEndDate) VALUES (%s, %s, %s, %s)"


In [16]:
count_rows(table)

Total number of rows  [(100,)]


100

In [ ]:
delete_all_rows(table)

In [ ]:
# Data generation

query = "INSERT INTO TaxiTest.insurance (idInsurance, InsCompany_idInsCompany, InsuranceEndDate, InsuranceStartDate) VALUES (%s , %s , %s , %s )"
data = []

total_number = 100

for i in range(total_number):
    idInsurance = i + 1 
    InsCompany_idInsCompany = np.random.randint(1, 30)
    InsuranceStartDate = date(2014 + np.random.randint(-3, 3), np.random.randint(1, 10), np.random.randint(1, 25))
    InsuranceEndDate = InsuranceStartDate + timedelta(days=25*int(np.random.choice(np.array([6, 8, 12]))))
    data.append((idInsurance, InsCompany_idInsCompany, InsuranceStartDate, InsuranceEndDate))
    #print(data[i])
    
print("Q: ", query)
print("D: ", data[0])

insert_many_rows(table, query, data)

## Table `car`

*** Changes ***
~~~~python
# Modify type
cursor.execute("ALTER TABLE " + table + " MODIFY ChildSeat INT")
~~~~


In [ ]:
# Load the data from stored text file
# 1. Make json file
file = "Car/MarkModel.txt"
cars = set()
with codecs.open(file, 'r', encoding='utf-8') as f:
    for line in f:
        cars.add(line[:-2])

# Choose unique
cars = sorted(list(cars))
with open('Car/Cars.json', 'w') as outfile:
    json.dump(cars, outfile)


In [8]:
table = DB_NAME + "." + "car"
get_table_review(table)
count_rows(table)

Reviewing  TaxiTest.car
idCar, int(11), NO, PRI, None, 
Lisense, varchar(45), NO, , None, 
Mark, varchar(45), NO, , None, 
Model, varchar(45), NO, , None, 
Insurance_idInsurance, int(11), NO, MUL, None, 
ChildSeat, int(11), YES, , None, 

Columns: ( idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat )
query = "INSERT INTO TaxiTest.car (idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat) VALUES (%s, %s, %s, %s, %s, %s)"
Total number of rows  [(100,)]


100

In [ ]:
# Data generation
query = "INSERT INTO TaxiTest.car (idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat) VALUES (%s, %s, %s, %s, %s, %s)"
data = []

total_number = 100
# Get insurances (better in random order), we have 100 insurances -> total_number = 100 of cars available
# Warning: np.arange(a) return from 0 to a - 1
# Writing Insurance_idInsurance which doesn't exist in insurance is restricted - QUERY will not be executed
insurances = np.arange(1, total_number + 1)
np.random.shuffle(insurances)

# Load cars
with open("Car/Cars.json") as f:
    cars = json.load(f)

# Generate UNIQUE car licenses
licenses = set()
alphanums = string.ascii_uppercase + string.digits
while len(licenses) != 100:
    licenses.add(''.join(random.choice(alphanums) for _ in range(7)))
licenses = list(licenses)

for i in range(total_number):
    idCar = i + 1 
    Lisense = licenses[i]
    (Mark, Model) = np.random.choice(cars).split(' ')
    # Failed processing format-parameters; Python 'int32' cannot be converted to a MySQL type
    Insurance_idInsurance = int(insurances[i])
    ChildSeat = int(np.random.choice([0, 1]))
    data.append((idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat))
    # print(data[i])
    
print("Q: ", query)
print("D: ", data[0])

#insert_many_rows(table, query, data)

## Table `driver` 

*** Changes ***
~~~~python
# Add isWorking to check if driver working or not
cursor.execute("ALTER TABLE " + table + " ADD COLUMN isWorking INT NOT NULL")
cursor.execute("ALTER TABLE " + table + " MODIFY COLUMN Name VARCHAR(45) NOT NULL")
~~~~

In [96]:
table = DB_NAME + '.' + 'driver' 
get_table_review(table)
count_rows(table)

Reviewing  TaxiTest.driver
idDriver, int(11), NO, PRI, None, 
Name, varchar(45), NO, , None, 
DriverLisence, varchar(45), NO, UNI, None, 
Phone, varchar(45), NO, UNI, None, 
isWorking, int(11), NO, , None, 

Columns: ( idDriver, Name, DriverLisence, Phone, isWorking )
query = "INSERT INTO TaxiTest.driver (idDriver, Name, DriverLisence, Phone, isWorking) VALUES (%s, %s, %s, %s, %s)"
Total number of rows TaxiTest.driver: [(75,)].


75

In [91]:
delete_all_rows(table)

In [95]:
total_number = 75
query = "INSERT INTO TaxiTest.driver (idDriver, Name, DriverLisence, Phone, isWorking) VALUES (%s, %s, %s, %s, %s)"

# Get all possible names_surnames combinations
all_names_surnames = []
with codecs.open("AllCombinations.txt", 'r', encoding='utf-8') as f:
    for line in f:
        all_names_surnames.append(line[:-2])

# Generate UNIQUE driver licenses
licenses = set()
while len(licenses) != total_number:
    part1 = ''.join(random.choice(string.ascii_uppercase) for _ in range(7))
    part2 = ''.join(random.choice(string.digits) for _ in range(4))
    licenses.add(part1 + part2)
licenses = list(licenses)


# Get names
data = []
names = list(np.random.choice(all_names_surnames, size=total_number))
for i in range(total_number):
    idDriver = i + 1
    # Failed processing format-parameters; Python 'str_' cannot be converted to a MySQL type
    Name = str(names[i])
    DriverLisence = licenses[i]
    Phone = '+7({}){:07}'.format(np.random.randint(50, 990), np.random.randint(0, 9999999))
    isWorking = int(np.random.choice([0, 1], p=[0.15, 0.85]))
    data.append((idDriver, Name, DriverLisence, Phone, isWorking))
    
print("Q: ", query)
print("D: ", data[0])

insert_many_rows(table, query, data)

Q:  INSERT INTO TaxiTest.driver (idDriver, Name, DriverLisence, Phone, isWorking) VALUES (%s, %s, %s, %s, %s)
D:  (1, 'Роберт Моисеев', 'GJUQZRR1063', '+7(296)5290592', 1)
No result set to fetch from.


## Table `address`

*** Changes ***
~~~~python
# Drop GeoPoint
cursor.execute("ALTER TABLE " + table + " DROP COLUMN GeoPoint")
# Delete index on UNIQUE city
cursor.execute("ALTER TABLE " + table + " DROP INDEX City_UNIQUE")
~~~~

In [ ]:
table = DB_NAME + '.' + 'address' 
get_table_review(table)
count_rows(table)

In [ ]:
delete_all_rows(table)

In [ ]:
total_number = 1000
query = "INSERT INTO TaxiTest.address (idAddress, City, Street, HouseNumber) VALUES (%s, %s, %s, %s)"

City = "Kazan"
# Get all streets
with open('Streets.json') as f:
    streets = json.load(f)
    streets = np.random.choice(streets, total_number, replace=False)

# Get names
data = []
for i in range(total_number):
    idAddress = i + 1
    Street = str(streets[i])
    HouseNumber = int(random.randint(1, 100))
    data.append((idAddress, City, Street, HouseNumber))
    
print("Q: ", query)
print("D: ", data[0])

insert_many_rows(table, query, data)

## Table `taxi`

*** Changes ***
~~~~python
# Rename
cursor.execute("ALTER TABLE " + table + " CHANGE LatUpdate LastUpdate DATETIME")
# Change datatatype in case Python won`t permit writing to TINYINT
cursor.execute("ALTER TABLE " + table + " MODIFY COLUMN Available INT")

# Change to AUTO-INCREMENT
cursor.execute("ALTER TABLE " + table + " MODIFY COLUMN idTaxi INT AUTO_INCREMENT NOT NULL")
~~~~
*** Late: cannot change column 'idtaxi': used in a foreign key constraint 'fk_Order_Taxi' of table 'taxitest.order' ***

In [133]:
table = DB_NAME + '.' + 'Driver'
get_table_review(table, True)
table = DB_NAME + '.' + 'Car'
get_table_review(table, True)

table = "Address"
address_rows_amount = count_rows("Address")
print("")

# Try to select values from tables
table = DB_NAME + '.' + 'Taxi'
get_table_review(table)
address_rows_amount = count_rows(table)

Reviewing  TaxiTest.Driver

Columns: ( idDriver, Name, DriverLisence, Phone, isWorking )
query = "INSERT INTO TaxiTest.Driver (idDriver, Name, DriverLisence, Phone, isWorking) VALUES (%s, %s, %s, %s, %s)"
Reviewing  TaxiTest.Car

Columns: ( idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat )
query = "INSERT INTO TaxiTest.Car (idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat) VALUES (%s, %s, %s, %s, %s, %s)"
Total number of rows Address: [(1000,)].

Reviewing  TaxiTest.Taxi
idTaxi, int(11), NO, MUL, None, 
Driver_idDriver, int(11), NO, PRI, None, 
Car_idCar, int(11), NO, PRI, None, 
Current_idAddress, int(11), NO, MUL, None, 
Available, int(11), YES, , None, 
LastUpdate, datetime, YES, , None, 

Columns: ( idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LastUpdate )
query = "INSERT INTO TaxiTest.Taxi (idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LastUpdate) VALUES (%s, %s, %s, %s, %s, %s)"
Total number of rows TaxiTest.Taxi

In [141]:
delete_all_rows(table)

*** If driver is not working -> taxi is not available ***

*** Taxi may be not available even if driver is not deleted *** 

In [143]:
table = "Taxi"

# Using Taxi table attributes order to create query.
# INSERT INTO Taxi (idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LatUpdate)
address_rows_amount = count_rows("Address")
q = "SELECT idDriver, isWorking, idCar, ROUND(1 + RAND() * %s) as Current_idAddress FROM Driver JOIN Car " % (address_rows_amount - 1)

total_amount = 500
result = select_rows(table, q)

taxi_combinations_set = np.arange(1, len(result))
np.random.shuffle(taxi_combinations_set)                                  
                                  
q = "INSERT INTO TaxiTest.Taxi (idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LastUpdate) VALUES (%s, %s, %s, %s, %s, %s)"
data = []

for i in range(total_amount):
    row = result[taxi_combinations_set[i]]
    isWorking = row[1]
    LastUpdate = datetime.now() + timedelta(hours=np.random.randint(1, 12), minutes=np.random.randint(5, 60))
    if isWorking == 0:
        LastUpdate -= timedelta(days=np.random.randint(4, 90))
    available = int(np.random.choice([0, 1], p=[0.4, 0.6]) if isWorking else 0)
    # Order: (idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LastUpdate)
    data.append((i + 1, int(row[0]), int(row[2]), int(row[3]), available, LastUpdate))
    #print(data[i])
    
insert_many_rows(table, q, data)

Total number of rows Address: [(1000,)].
No result set to fetch from.


## Table `Operator`

In [145]:
table = DB_NAME + '.' + 'operator' 
get_table_review(table)
count_rows(table)

Reviewing  TaxiTest.operator
idOperator, int(11), NO, PRI, None, 
Name, varchar(45), NO, , None, 
Phone, varchar(45), NO, UNI, None, 
Mail, varchar(45), NO, , None, 

Columns: ( idOperator, Name, Phone, Mail )
query = "INSERT INTO TaxiTest.operator (idOperator, Name, Phone, Mail) VALUES (%s, %s, %s, %s)"
Total number of rows TaxiTest.operator: [(0,)].


0

In [ ]:
delete_all_rows(table)

In [156]:
total_number = 50
query = "INSERT INTO TaxiTest.operator (idOperator, Name, Phone, Mail) VALUES (%s, %s, %s, %s)"

# Get all possible names_surnames combinations
names = []
with codecs.open("AllCombinations.txt", 'r', encoding='utf-8') as f:
    for line in f:
        names.append(line[:-2])

names = np.random.choice(names, total_number)

# Generate UNIQUE phones
phones = set()
while len(phones) != total_number:
    phone = '+7(843)' + '{:07}'.format(np.random.randint(0, 9999999))
    phones.add(phone)
phones = list(phones)

symbols = (u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ",
           u"abvgdeejzijklmnoprstufhzcss_y_euaABVGDEEJZIJKLMNOPRSTUFHZCSS_Y_EUA")
tr = {ord(a):ord(b) for a, b in zip(*symbols)}

data = []
for i in range(total_number):
    idOperator = i + 1
    name = names[i].replace(' ', '.')
    mail = name.translate(tr).lower() + '@gmail.com'
    data.append((idOperator, str(names[i]), str(phones[i]), str(mail)))
    
print("Q: ", query)
print("D: ", data[0])

# insert_many_rows(table, query, data)

Q:  INSERT INTO TaxiTest.operator (idOperator, Name, Phone, Mail) VALUES (%s, %s, %s, %s)
D:  (1, 'Геннадий Игнатов', '+7(843)3240492', 'gennadij.ignatov@gmail.com')


## Table `Client`

*** Changes ***
~~~~python
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN Name Name VARCHAR(45) AFTER idClient")
~~~~

In [190]:
table = DB_NAME + '.' + 'client' 
get_table_review(table)
count_rows(table)

Reviewing  TaxiTest.client
idClient, int(11), NO, PRI, None, auto_increment
Name, varchar(45), YES, , None, 
Phone, varchar(45), NO, UNI, None, 
Password, varchar(45), NO, , None, 

Columns: ( idClient, Name, Phone, Password )
query = "INSERT INTO TaxiTest.client (idClient, Name, Phone, Password) VALUES (%s, %s, %s, %s)"
Total number of rows TaxiTest.client: [(0,)].


0

In [189]:
delete_all_rows(table)

In [191]:
total_number = 5000
query = "INSERT INTO TaxiTest.client (idClient, Name, Phone, Password) VALUES (%s, %s, %s, %s)"

# Get all possible names_surnames combinations
names = []
with codecs.open("AllCombinations.txt", 'r', encoding='utf-8') as f:
    for line in f:
        names.append(line[:-2])

names = np.random.choice(names, total_number, replace=False)

# Generate UNIQUE phones
phones = set()
while len(phones) != total_number:
    phone = '+7({:03}){:07}'.format(np.random.randint(50, 990), np.random.randint(0, 9999999))
    phones.add(phone)
phones = list(phones)

alphanums = string.ascii_letters + string.digits
    
data = []
for i in range(total_number):
    idClient = i + 1
    password = ''.join(random.choice(alphanums) for _ in range(7))
    data.append((idClient, str(names[i]), phones[i], password))
    
print("Q: ", query)
print("D: ", data[0])

# insert_many_rows(table, query, data)

Q:  INSERT INTO TaxiTest.client (idClient, Name, Phone, Password) VALUES (%s, %s, %s, %s)
D:  (1, 'Геннадий Устинов', '+7(258)5810388', '3J49E1e')
No result set to fetch from.


## Table `PhoneCall`

*** Changes ***
~~~~python
# Unnset NOT NULL
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN `Duration` `Duration` INT AFTER Datetime")
# Change Primary Key from (idOperator, idClient) to idCall
cursor.execute("ALTER TABLE " + table + " DROP PRIMARY KEY, ADD PRIMARY KEY (idCall)")

# Rename to remove MySQL Errors
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN `datetime` `datetimeValue` DATETIME")
# Rename because Call is a system word
cursor.execute("ALTER TABLE " + table + " RENAME TO PhoneCall")
~~~~
*** Change idCall to NOT AUTO-INCREMENT ***
~~~~sql
SHOW CREATE TABLE PhoneCall;
ALTER TABLE `Order` DROP FOREIGN KEY fk_Order_Call;
ALTER TABLE `PhoneCall` CHANGE COLUMN `idCall` `idCall` INT NOT NULL;
# Adding PRIMARY KEY on idCall not needed in case it already exists 

# Return CONSTRAINT (Workbench): Used inital Create script to identify constraint and FK names.
ALTER TABLE `Order` ADD CONSTRAINT fk_Order_Call FOREIGN KEY (`Call_idCall`) REFERENCES `PhoneCall`(`idCall`);

# Check constraint
SELECT TABLE_NAME,COLUMN_NAME,CONSTRAINT_NAME, REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME
FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
WHERE
  REFERENCED_TABLE_SCHEMA = 'TaxiTest' AND
  REFERENCED_TABLE_NAME = 'PhoneCall' AND
  REFERENCED_COLUMN_NAME = 'idCall';

~~~~
*** Temporal change ***
~~~~python
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN `Datetime` `Datetime` Datetime AFTER Client_idClient")
~~~~
*** Roollback (AFTER THE COMPLETING 'Order') ***
~~~~python
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN `Datetime` `Datetime` Datetime NOT NULL AFTER Client_idClient")
~~~~

In [9]:
table = DB_NAME + '.' + 'PhoneCall'
get_table_review(table)
count_rows(table)

TaxiTest.PhoneCall
Field, Type, Null, Key, Default, Extra

idCall, int(11), NO, PRI, None, 
Operator_idOperator, int(11), NO, MUL, None, 
Client_idClient, int(11), NO, MUL, None, 
datetimeValue, datetime, YES, , None, 
Duration, int(11), YES, , None, 

Columns: ( idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration )
query = "INSERT INTO TaxiTest.PhoneCall (idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration) VALUES (%s, %s, %s, %s, %s)"
Total number of rows TaxiTest.PhoneCall: [(1500,)].


1500

In [8]:
delete_all_rows(table)

In [7]:
'''
query = "SELECT idOperator, idClient FROM Operator JOIN Client"
result = select_rows(table, q)
client_max_id * operators_max_id = len(result)
'''
# This query is hard, cause handles 5000 * 50 combinations -> 
# execute with sumpler queries (need only range of corresponding ids)

operators_max_id = count_rows("Operator")
client_max_id = count_rows("Client")

total_amount = 1500
all_combinations = list(itertools.product(np.arange(1, operators_max_id), np.arange(1, client_max_id)))
np.random.shuffle(all_combinations)

# Roolback on datetime then
query = "INSERT INTO PhoneCall (idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration) VALUES (%s, %s, %s, %s, %s)"
data = []


print("Q: ", query)
with open('C:/SQL/PhoneCall - INSERT 1500 with DATETIME.sql', 'w') as f:
    f.write('INSERT INTO PhoneCall (idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration) VALUES \n')
    for i in range(total_amount):
        idCall = i + 1
        (Operator_idOperator, Client_idClient) = all_combinations[i]
        duration = int(np.random.randint(20, 120))
        # Order: Operator_idOperator, Client_idClient, Duration  
        datetime_value = datetime(2018, np.random.randint(1, 8), np.random.randint(1, 28)) + timedelta(minutes=np.random.randint(-1000, 1000))
        formatted_date = datetime_value.strftime('%Y-%m-%d %H:%M:%S')
        data.append((idCall, Operator_idOperator, Client_idClient, formatted_date, duration))
        f.write("(%s, %s, %s, \'%s\', %s),\n" % data[i])
    
print("Q: ", query)
print("D: ", data[0])
# insert_many_rows(table, query, data)

Total number of rows Operator: [(50,)].
Total number of rows Client: [(5000,)].
Q:  INSERT INTO PhoneCall (idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration) VALUES (%s, %s, %s, %s, %s)
Q:  INSERT INTO PhoneCall (idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration) VALUES (%s, %s, %s, %s, %s)
D:  (1, 46, 1087, '2018-06-15 04:29:00', 56)


*** Result: *** written to text file and inserted using MySQL Workbench. 

## Table `PaymentService`

In [6]:
table = DB_NAME + '.' + 'PaymentService'
get_table_review(table)
count_rows(table)

TaxiTest.PaymentService
Field, Type, Null, Key, Default, Extra

idPaymentService, int(11), NO, PRI, None, 
Name, varchar(45), NO, UNI, None, 
Address, varchar(45), NO, UNI, None, 
Site, varchar(45), NO, UNI, None, 

Columns: ( idPaymentService, Name, Address, Site )
query = "INSERT INTO TaxiTest.PaymentService (idPaymentService, Name, Address, Site) VALUES (%s, %s, %s, %s)"
Total number of rows TaxiTest.PaymentService: [(0,)].


0

In [42]:
delete_all_rows(table)

In [53]:
with open('PaymentService/banks.json', 'r') as f:
    services_list = json.load(f)
total_amount = len(services_list)

with open('Streets.json', 'r') as f:
    streets = json.load(f)
    streets = np.random.choice(streets, total_amount)
    

    
query = "INSERT INTO `paymentservice` (idPaymentService, Name, Address, Site) VALUES (%s, %s, %s, %s)"
print(query)
data = []    
for i in range(total_amount):
    idPaymentService = i + 1
    Name = services_list[i]
    Address = 'Kazan, ' + streets[i] + ', ' + str(np.random.randint(1, 10))
    lower = Name.lower() 
    Site = lower.split('payservice')[0] + '.com'    
    data.append((idPaymentService, Name, Address, Site))
    
print("Q: ", query)
print("D: ", data[0])
print("")

insert_many_rows(table, query, data)

INSERT INTO `paymentservice` (idPaymentService, Name, Address, Site) VALUES (%s, %s, %s, %s)
Q:  INSERT INTO `paymentservice` (idPaymentService, Name, Address, Site) VALUES (%s, %s, %s, %s)
D:  (1, 'SberbankPayService', 'Kazan, Meridian, 9', 'sberbank.com')

You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'TaxiTest.PaymentService' at line 1


~~~~sql
INSERT INTO TaxiTest.PaymentService (idPaymentService, Name, Address, Site) VALUES
(1, 'SberbankPayService', 'Kazan, Baltic, 4', 'sberbank.com') ,
(2, 'VTBPayService', 'Kazan, Peter, 2', 'vtb.com') ,
(3, 'GazprombankPayService', 'Kazan, Kirov, 6', 'gazprombank.com') ,
(4, 'VTB-24PayService', 'Kazan, Ichigovaya, 8', 'vtb-24.com') ,
(5, 'RosselkhozbankPayService', 'Kazan, Syzranskaya, 8', 'rosselkhozbank.com') ,
(6, 'BankPayService', 'Kazan, Aktaş, 1', 'bank.com') ,
(7, 'AlfaPayService', 'Kazan, Winter, 1', 'alfa.com') ,
(8, 'NomosPayService', 'Kazan, Armuti, 8', 'nomos.com') ,
(9, 'UnicreditPayService', 'Kazan, Cross-Road, 1', 'unicredit.com') ,
(10, 'RosbankPayService', 'Kazan, Kyrlay, 9', 'rosbank.com') ,
(11, 'PromsvyazbankPayService', 'Kazan, Snowball, 2', 'promsvyazbank.com') ,
(12, 'RaiffeisenbankPayService', 'Kazan, Bastion, 7', 'raiffeisenbank.com');
~~~~


## Table `Payment`

*** Changes ***
~~~~python
# Change to Cash (value 0) or Card (value 1)
cursor.execute("ALTER TABLE Payment MODIFY Type INT NOT NULL AFTER PayTime")

# Drop FOREIGN KEY 
cursor.execute("ALTER TABLE Payment DROP FOREIGN KEY fk_Payment_PaymentService1")

# Change COLUMN PaymentService_idPaymentService for NULL values insertion opportunity
cursor.execute("ALTER TABLE Payment CHANGE PaymentService_idPaymentService PaymentService_idPaymentService INT AFTER PayTime")

# Return the KEY, now available to insert with NULL values
cursor.execute("ALTER TABLE Payment ADD FOREIGN KEY fk_Payment_PaymentService1 (PaymentService_idPaymentService) REFERENCES PaymentService(idPaymentService) ")

# Ruble coins are not used at all, DOUBLE type is not necessary 
cursor.execute("ALTER TABLE Payment MODIFY Cost INT NOT NULL ")

# Rename to exclude mistakes
cursor.execute("ALTER TABLE Payment CHANGE Type PaymentType INT NOT NULL ")

# Delete Cost Column cause it is already in the Order
cursor.execute("ALTER TABLE Payment DROP COLUMN COST")
~~~~
~~~~sql
# Add explicitly one row to table PaymentService with idPaymentService = 0
INSERT INTO `paymentservice` (idPaymentService, Name, Address, Site) VALUES (0, 'CashService', 'NoAddress', 'NoSite');

~~~~

In [21]:
table = DB_NAME + '.' + 'Payment'
get_table_review(table)
count_rows(table)

TaxiTest.Payment
Field, Type, Null, Key, Default, Extra

idPayment, int(11), NO, PRI, None, auto_increment
PayTime, datetime, YES, , None, 
PaymentType, int(11), NO, , None, 
PaymentService_idPaymentService, int(11), YES, MUL, None, 
Paycheck, varchar(45), YES, , None, 

Columns: ( idPayment, PayTime, PaymentType, PaymentService_idPaymentService, Paycheck )
query = "INSERT INTO TaxiTest.Payment (idPayment, PayTime, PaymentType, PaymentService_idPaymentService, Paycheck) VALUES (%s, %s, %s, %s, %s)"
Total number of rows TaxiTest.Payment: [(0,)].


0

In [19]:
delete_all_rows(table)

## Table `Orders`

*** Changes ***
~~~~python
# Order is a keyword
cursor.execute("ALTER TABLE Taxitest.Order RENAME Orders")
~~~~

In [10]:
table = DB_NAME + '.' + 'Orders'
get_table_review(table)
count_rows(table)

TaxiTest.Orders
Field, Type, Null, Key, Default, Extra

idOrder, int(11), NO, PRI, None, 
Call_idCall, int(11), NO, MUL, None, 
Departure_idAddress, int(11), NO, MUL, None, 
Destination_idAddress, int(11), NO, MUL, None, 
Taxi_idTaxi, int(11), NO, MUL, None, 
Payment_idPayment, int(11), NO, MUL, None, 
StartDatetime, datetime, NO, , None, 
FinishDatetime, datetime, YES, , None, 
Cost, int(11), NO, , None, 
Rate, int(11), YES, , None, 

Columns: ( idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, Payment_idPayment, StartDatetime, FinishDatetime, Cost, Rate )
query = "INSERT INTO TaxiTest.Orders (idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, Payment_idPayment, StartDatetime, FinishDatetime, Cost, Rate) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
Total number of rows TaxiTest.Orders: [(0,)].


0

In [68]:
delete_all_rows(table)

In [3]:
# Try to extract/insert datetime values
q = "SELECT idCall, datetimeValue FROM PhoneCall"
phone_call_data = select_rows(q)
for i in range(len(phone_call_data)):
    print(phone_call_data[i])
    t = phone_call_data[i][1]
    print(t + timedelta(minutes=50))
    break

(1, datetime.datetime(2018, 6, 15, 4, 29))
2018-06-15 05:19:00


In [25]:
from numpy.random import randint
from numpy.random import choice

MAX_PAYMENT_SERVICE_ID = count_rows("PaymentService")
MAX_CALL_ID = count_rows('PhoneCall')
MAX_ADDRESS_ID = count_rows('Address')
MAX_TAXI_ID = count_rows('Taxi')

TOTAL_ORDERS_AMOUNT = 1200

order_from_call = choice(np.arange(1, len(phone_call_data)), TOTAL_ORDERS_AMOUNT, replace=False)
order_from_call = sorted(order_from_call)

f_order = open('C:/SQL/Orders.sql', 'w')    
query_order = "INSERT INTO Orders (idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, Payment_idPayment, StartDatetime, FinishDatetime, Cost, Rate) VALUES\n"
f_order.write(query_order)
data_order = []

f_payment = open('C:/SQL/Payment.sql', 'w')
query_payment = "INSERT INTO Payment (idPayment, PaymentType, PaymentService_idPaymentService, Paycheck) VALUES \n"
f_payment.write(query_payment)
data_payment = []

q = "SELECT idCall, datetimeValue FROM PhoneCall ORDER BY datetimeValue"
phone_call_data = select_rows(q)

for i in range(TOTAL_ORDERS_AMOUNT):
    idOrder = i + 1
    idPayment = i + 1
    t = order_from_call[i]
    Call_idCall = phone_call_data[t][0]
    StartDatetime = phone_call_data[t][1] + timedelta(minutes=randint(5, 360))

    (Departure_idAddress, Destination_idAddress) = choice(np.arange(1, MAX_ADDRESS_ID), 2, replace=False) 
    Taxi_idTaxi = randint(1, MAX_TAXI_ID)
    FinishDatetime = StartDatetime + timedelta(minutes=randint(5, 45))
    Cost = int(6 * (FinishDatetime - StartDatetime).total_seconds() // 60)
    Rate = choice([3,4,5], p=[0.2, 0.3, 0.5])
    
    order_tuple = (idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, idPayment, StartDatetime, FinishDatetime, Cost, Rate)
    row_order = "(%s, %s, %s, %s, %s, %s, \'%s\',\'%s\', %s, %s),\n"
 
    # Payment
    payment_type = choice([0, 1], p=[0.65, 0.35]) # ['Card', 'Cash']
    if payment_type == 0:
        id_payment_service = randint(1, MAX_PAYMENT_SERVICE_ID)
        paycheck = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10))
        row_payment = "(%s, %s, %s, \"%s\"),\n"
    else:
        id_payment_service = 'NULL'
        paycheck = 'NULL'
        row_payment = "(%s, %s, %s, %s),\n"
    payment_tuple = (idPayment, payment_type, id_payment_service, paycheck)
    
    if i == TOTAL_ORDERS_AMOUNT - 1:
        row_order = row_order[:-2] + ';'
        row_payment = row_payment[:-2] + ';'
    
    f_order.write(row_order % order_tuple)
    f_payment.write(row_payment % payment_tuple)
    
f_order.close()
f_payment.close()
        

print("\nQ: %sD: %s" % (query_order, order_tuple))
print("\nQ: %sD: %s" % (query_payment, payment_tuple))

Total number of rows PaymentService: [(13,)].
Total number of rows PhoneCall: [(1500,)].
Total number of rows Address: [(1000,)].
Total number of rows Taxi: [(500,)].

Q: INSERT INTO Orders (idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, Payment_idPayment, StartDatetime, FinishDatetime, Cost, Rate) VALUES
D: (1200, 688, 68, 192, 415, 1200, datetime.datetime(2018, 7, 27, 19, 47), datetime.datetime(2018, 7, 27, 19, 57), 60, 3)

Q: INSERT INTO Payment (idPayment, PaymentType, PaymentService_idPaymentService, Paycheck) VALUES 
D: (1200, 0, 7, 'WDQ9T8C4H9')


# SQL запросы

### Список всех свободных авто

In [ ]:
q = ""

### Список всех клиентов, которых обслуживал авто (входной параметр – номер Авто)

### Список всех авто, которыми пользовался клиент в интервале времени (входные параметры – телефон клиента, дата начала и дата окончания)

### Список всех заказов, которые обработаны оператором (входной параметр id-оператора) 


In [78]:
cursor.close()
cnx.close()